In [0]:
#!tar -xvzf LC82071142014003LGN00.tar

training_set_path='SegNet/Landsat/train.txt'
val_set_path='SegNet/Landsat/val.txt'
test_set_path='SegNet/CamVid/test.txt'
model_path='models/'

num_classes=2
image_shape=(512,512,10)
padding=((0,0),(0,0))
batch_size=5
epochs=100
training_set_size=153
val_set_size=30
test_set_size=233
optimizer='sgd'
loss_function='categorical_crossentropy'
metrics=['accuracy']


In [0]:
from keras.layers import Activation,BatchNormalization,Conv2D
from keras.engine.topology import Layer
#import keras.backend as K
#import tensorflow as tf
import keras.backend.tensorflow_backend as K

class MaxPoolingWithIndices(Layer):
    def __init__(self, pool_size,strides,padding='SAME',**kwargs):
        super(MaxPoolingWithIndices, self).__init__(**kwargs)
        self.pool_size=pool_size
        self.strides=strides
        self.padding=padding
        return
    def call(self,x):
        pool_size=self.pool_size
        strides=self.strides
        if isinstance(pool_size,int):
            ps=[1,pool_size,pool_size,1]
        else:
            ps=[1,pool_size[0],pool_size[1],1]
        if isinstance(strides,int):
            st=[1,strides,strides,1]
        else:
            st=[1,strides[0],strides[1],1]
        output1,output2=K.tf.nn.max_pool_with_argmax(x,ps,st,self.padding)
        return [output1,output2]
    def compute_output_shape(self, input_shape):
        if isinstance(self.pool_size,int):
            output_shape=(input_shape[0],input_shape[1]//self.pool_size,input_shape[2]//self.pool_size,input_shape[3])
        else:
            output_shape=(input_shape[0],input_shape[1]//self.pool_size[0],input_shape[2]//self.pool_size[1],input_shape[3])
        return [output_shape,output_shape]


class UpSamplingWithIndices(Layer):
    def __init__(self, **kwargs):
        super(UpSamplingWithIndices, self).__init__(**kwargs)
        return
    def call(self,x):
        argmax=K.cast(K.flatten(x[1]),'int32')
        max_value=K.flatten(x[0])
        with K.tf.variable_scope(self.name):
            input_shape=K.shape(x[0])
            batch_size=input_shape[0]
            image_size=input_shape[1]*input_shape[2]*input_shape[3]
            output_shape=[input_shape[0],input_shape[1]*2,input_shape[2]*2,input_shape[3]]
            indices_0=K.flatten(K.tf.matmul(K.reshape(K.tf.range(batch_size),(batch_size,1)),K.ones((1,image_size),dtype='int32')))
            indices_1=argmax%(image_size*4)//(output_shape[2]*output_shape[3])
            indices_2=argmax%(output_shape[2]*output_shape[3])//output_shape[3]
            indices_3=argmax%output_shape[3]
            indices=K.tf.stack([indices_0,indices_1,indices_2,indices_3])
            output=K.tf.scatter_nd(K.transpose(indices),max_value,output_shape)
            return output
    def compute_output_shape(self, input_shape):
        return input_shape[0][0],input_shape[0][1]*2,input_shape[0][2]*2,input_shape[0][3]

def CompositeConv(inputs,num_layers,num_features):
    output=inputs
    if isinstance(num_features,int):
        for i in range(num_layers):
            output=Conv2D(num_features,(7,7),padding='same')(output)
            output=BatchNormalization(axis=3)(output)
            output=Activation('relu')(output)
        return output
    for i in range(num_layers):
        output=Conv2D(num_features[i],(7,7),padding='same')(output)
        output=BatchNormalization(axis=3)(output)
        output=Activation('relu')(output)
    return output


In [0]:
from keras.models import Model
from keras.layers import Activation,Input,ZeroPadding2D,Cropping2D
#from custom_layers import MaxPoolingWithIndices,UpSamplingWithIndices,CompositeConv
#import config as cf

def create_model():
    inputs=Input(shape=image_shape)

    x = ZeroPadding2D(padding)(inputs)

    x=CompositeConv(x,2,64)
    x,argmax1=MaxPoolingWithIndices(pool_size=2,strides=2)(x)
    
    x=CompositeConv(x,2,64)
    x,argmax2=MaxPoolingWithIndices(pool_size=2,strides=2)(x)
    
    x=CompositeConv(x,3,64)
    x,argmax3=MaxPoolingWithIndices(pool_size=2,strides=2)(x)

    x=CompositeConv(x,3,64)
    x,argmax4=MaxPoolingWithIndices(pool_size=2,strides=2)(x)

    x=CompositeConv(x,3,64)
    x,argmax5=MaxPoolingWithIndices(pool_size=2,strides=2)(x)

    x=UpSamplingWithIndices()([x,argmax5])
    x=CompositeConv(x,3,64)

    x=UpSamplingWithIndices()([x,argmax4])
    x=CompositeConv(x,3,64)

    x=UpSamplingWithIndices()([x,argmax3])
    x=CompositeConv(x,3,64)

    x=UpSamplingWithIndices()([x,argmax2])
    x=CompositeConv(x,2,64)
    
    x=UpSamplingWithIndices()([x,argmax1])
    x=CompositeConv(x,2,[64,num_classes])

    x=Activation('softmax')(x)

    y=Cropping2D(padding)(x)
    my_model=Model(inputs=inputs,outputs=y)
    
    return my_model

In [0]:
import imageio
import numpy as np
from keras.utils import to_categorical
#import config as cf

def read_image_batch(data_path,batch_size):
    while True:
        image_list=open(data_path).readlines()
        l=len(image_list)
        
        num_batch=l//batch_size
        if num_batch*batch_size<l:
            num_batch+=1
        for i in range(num_batch):
            batch_set=image_list[batch_size*i:min(batch_size*(i+1),l)]
            batch_set=[bs.strip().split() for bs in batch_set]
            #X=np.array([imageio.imread(line[0][1:]) for line in batch_set])
            #labels=np.array([imageio.imread(line[1][1:]) for line in batch_set])
            X=np.array([np.load(line[0][1:]) for line in batch_set])
            labels=np.array([np.load(line[1][1:]) for line in batch_set])
            y=to_categorical(labels,num_classes)
           
            yield tuple((X, y))


In [0]:
import argparse
from keras.utils import multi_gpu_model
from keras.models import Model
from keras.layers import Activation,Input
#from custom_layers import MaxPoolingWithIndices,UpSamplingWithIndices,CompositeConv
#import config as cf
#from util import read_image_batch
#from create_model import create_model

def main(args):
    my_model=create_model()
    my_model.compile(optimizer,loss=loss_function,metrics=metrics)
    
    if args['resume']:
        my_model.load_weights(model_path+args['load'])
    
    val_data=read_image_batch(val_set_path, batch_size)
    train_data=read_image_batch(training_set_path,batch_size)
    print("read image batch")
    my_model.fit_generator(train_data,
                           steps_per_epoch=(training_set_size+1)//batch_size,
                           epochs=epochs,validation_data=val_data,
                           validation_steps=(val_set_size+1)//batch_size)
    print("model fit")
    my_model.save_weights(model_path+args['save'])

#parser = argparse.ArgumentParser()
#parser.add_argument("--save",default='my_model')
#parser.add_argument("--resume",action='store_true')
#parser.add_argument("--load",default='my_model')
args = {"save": "my_model", "resume":False, "load": "my_model"}
main(args)
